In [1]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split # 导入 train_test_split
from sklearn.preprocessing import StandardScaler
from PIL import Image
from tqdm import tqdm
import warnings

# 确保 PIL 版本兼容 (用于 resize)
try:
    # PIL 9.0.0+
    LANCZOS_RESAMPLE = Image.Resampling.LANCZOS
except AttributeError:
    # Older PIL
    LANCZOS_RESAMPLE = Image.LANCZOS

# --- 配置区域 ---

# *** 1. 路径已根据您的要求修改 ***
DATA_ROOT = "/mnt/hdd/jiazy/CelebA"
ATTR_FILE = os.path.join(DATA_ROOT, "list_attr_celeba.csv")
BASE_IMAGE_DIR = os.path.join(DATA_ROOT, "img_align_celeba")

# 保存处理后文件的输出目录
OUTPUT_DIR = "./features"

# --- 关键更改 ---
# 目标标签
LABEL_COLUMN = 'Attractive' 
# 新增：图像尺寸
IMAGE_SIZE = 224

# --------------------

def preprocess_celeba_flow(attr_file, base_image_dir, output_dir, label_column):
    """
    根据用户新需求 (8:1:1 划分, CSV加载, 224x224 resize) 预处理 CelebA
    """
    print(f"开始执行 CelebA 预处理流程 (标签: {label_column})...")
    os.makedirs(output_dir, exist_ok=True)
    
    # --- 1. 📖 加载数据 (Step 1) ---
    # *** 2. 已修改：加载 CSV 而不是 TXT，并且不再加载 partition 文件 ***
    print(" 正在加载标注文件 (list_attr_celeba.csv)...")
    try:
        # 加载 .csv 文件
        df_master = pd.read_csv(ATTR_FILE)
    except FileNotFoundError as e:
        print(f"错误：找不到文件 {e.filename}。")
        print("请确保 DATA_ROOT 和 ATTR_FILE 设置正确。")
        return
    
    # 原始脚本的 partition 和 merge 部分已被移除
    print(f" > 成功加载 {len(df_master)} 条记录。")

    # --- 2. 📝 定义特征 (Step 2) ---
    print(" 正在定义特征...")
    
    # *** 3. 已修复：初始化为空列表 ***
    continuous_cols = []
    
    # 类别特征是 40 个属性
    # 我们从 'image_id' 中排除
    # (假设 'image_id' 是 .csv 的第一列)
    categorical_cols = df_master.columns.drop(['image_id']).tolist()
    
    # 确保我们选择的标签在类别特征列表中（后续会移除）
    if label_column not in categorical_cols:
        print(f"错误：标签 '{label_column}' 不在属性列表中。")
        print(f"可用属性：{categorical_cols}")
        return
        
    # 定义非特征列
    # *** 4. 已修改：移除 'split' ***
    non_feature_cols = ['image_id', label_column]
    
    print(f" > 已定义 {len(continuous_cols)} 个连续特征。")
    print(f" > 已定义 {len(categorical_cols)} 个类别特征 (包含标签)。")
    print(f" > 目标标签 (Label): {label_column}")

    # --- 8. 🔪 数据集划分 (Step 8) ---
    # *** 5. 已彻底修改：使用 8:1:1 分层划分，替换原有的固定划分 ***
    print(f" 正在按 8:1:1 对 {label_column} 进行分层划分...")
    
    # 首先，分层划分出 80% 的训练集
    train_df, temp_df = train_test_split(
        df_master,
        test_size=0.2,
        stratify=df_master[label_column],
        random_state=42
    )
    
    # 然后，将剩余的 20% 平分为 10% 验证集和 10% 测试集
    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.5,
        stratify=temp_df[label_column],
        random_state=42
    )
    
    print(f" > 训练集: {len(train_df)} 样本")
    print(f" > 验证集: {len(val_df)} 样本")
    print(f" > 测试集: {len(test_df)} 样本")

    # --- 3. 🧹 特征筛选 (Step 3) ---
    # (此步骤逻辑在新的划分上依然有效)
    print(" 正在筛选恒定特征...")
    
    final_continuous_cols = list(continuous_cols)
    final_categorical_cols = list(categorical_cols)
    
    # (此部分代码不变，但现在作用于新划分的数据集)
    for col in continuous_cols:
        is_constant_train = train_df[col].nunique() <= 1
        is_constant_test = test_df[col].nunique() <= 1
        if is_constant_train and is_constant_test:
            print(f" > 移除恒定连续特征: {col}")
            final_continuous_cols.remove(col)
            
    for col in categorical_cols:
        is_constant_train = train_df[col].nunique() <= 1
        is_constant_test = test_df[col].nunique() <= 1
        if is_constant_train and is_constant_test:
            print(f" > 移除恒定类别特征: {col}")
            final_categorical_cols.remove(col)
            
    # 确保目标标签(label_column)不被视为特征
    if label_column in final_categorical_cols:
        final_categorical_cols.remove(label_column)
        print(f" > 已将 '{label_column}' 隔离为标签, 不作为特征。")
        
    print(f" > 剩余 {len(final_continuous_cols)} 个连续特征。")
    print(f" > 剩余 {len(final_categorical_cols)} 个类别特征 (表格属性)。")

    # --- 4. 🗺️ 路径处理 (Step 4) ---
    print(" 正在转换图像路径为绝对路径...")
    
    def create_absolute_path(image_id):
        # 假设 image_id 已经是 '000001.jpg' 这样的文件名
        return os.path.join(base_image_dir, image_id)

    train_df['absolute_image_path'] = train_df['image_id'].apply(create_absolute_path)
    val_df['absolute_image_path'] = val_df['image_id'].apply(create_absolute_path)
    test_df['absolute_image_path'] = test_df['image_id'].apply(create_absolute_path)
    
    non_feature_cols.extend(['absolute_image_path'])

    # --- 5. 🗑️ 数据清洗 (Step 5) ---
    print(" 正在过滤无效行 (基于路径和标签)...")
    initial_train, initial_val, initial_test = len(train_df), len(val_df), len(test_df)
    
    train_df = train_df.dropna(subset=['absolute_image_path', label_column])
    val_df = val_df.dropna(subset=['absolute_image_path', label_column])
    test_df = test_df.dropna(subset=['absolute_image_path', label_column])
    
    print(f" > 训练集移除 {initial_train - len(train_df)} 行")
    print(f" > 验证集移除 {initial_val - len(val_df)} 行")
    print(f" > 测试集移除 {initial_test - len(test_df)} 行")

    # --- 6. ✏️ 填充缺失值 (Step 6) ---
    print(" 正在处理缺失值/特殊编码...")

    # (连续特征部分跳过)
    for col in final_continuous_cols:
        mean_val = train_df[col].mean()
        train_df[col] = train_df[col].fillna(mean_val)
        val_df[col] = val_df[col].fillna(mean_val)
        test_df[col] = test_df[col].fillna(mean_val)
        
    # **CelebA 特定步骤**：
    # *** 6. 假设：我们假设 .csv 仍使用 -1/1 编码 ***
    # (如果您的 .csv 已是 0/1，可以注释掉 .replace(-1, 0) 这一行)
    print(" > [CelebA 特定] 转换属性：将 -1 映射到 0...")
    all_attr_cols = final_categorical_cols + [label_column]
    for col in all_attr_cols:
        train_df[col] = train_df[col].replace(-1, 0)
        val_df[col] = val_df[col].replace(-1, 0)
        test_df[col] = test_df[col].replace(-1, 0)
        # 无缺失值，不需要处理
        
    # --- 7. 🔢 特征工程 (类别) (Step 7) ---
    print(" 正在处理类别特征...")
    
    # *** 7. 已修复：初始化为空列表 ***
    field_lengths = []
    
    print(" > CelebA 属性已为 0/1 编码，跳过.cat.codes。")
    print(" > 正在计算基数 (Cardinalities)...")

    for col in final_categorical_cols:
        all_values = pd.concat([train_df[col], val_df[col], test_df[col]])
        cardinality = all_values.nunique()
        # 增加一个检查，以防 'MISSING' 真的被填充了
        if "MISSING" in all_values.unique():
            print(f" > 警告：特征 {col} 包含 'MISSING' 值。")
        field_lengths.append(cardinality)
        
    print(f" > {len(final_categorical_cols)} 个类别特征的基数 (部分): {field_lengths[:5]}...")
    
    # --- 9. 🔢 特征工程 (连续) (Step 9) ---
    print(" 正在标准化连续特征...")

    for _ in final_continuous_cols:
        field_lengths.insert(0, 1) 
    
    scaler = StandardScaler()
    
    if final_continuous_cols:
        scaler.fit(train_df[final_continuous_cols])
        train_df[final_continuous_cols] = scaler.transform(train_df[final_continuous_cols])
        val_df[final_continuous_cols] = scaler.transform(val_df[final_continuous_cols])
        test_df[final_continuous_cols] = scaler.transform(test_df[final_continuous_cols])
        print(" > 连续特征已标准化。")
    else:
        print(" > 没有连续特征，跳过标准化。")

    # --- 12. 🖼️ 图像处理 (JEPG 转 NPY) (Step 12) ---
    # *** 8. 已修改：增加 224x224 Resize 并修复 npy_path Bug ***
    print(f" 正在转换图像 (JPG -> NPY, 并 Resize 到 {IMAGE_SIZE}x{IMAGE_SIZE})...")

    def convert_jpg_to_npy(jpg_path):
        """
        打开 JPG, Resize, 转换为 NumPy 数组, 保存为.npy, 并返回.npy 路径
        """
        if not os.path.exists(jpg_path):
            warnings.warn(f" > 警告：找不到 JPG 文件 {jpg_path}。跳过。")
            return None
            
        # *** 9. 已修复：使用 [0] 来获取正确的文件基名 ***
        npy_path = os.path.splitext(jpg_path)[0] + ".npy"
        
        if os.path.exists(npy_path):
            return npy_path  # 跳过已存在的文件
            
        try:
            with Image.open(jpg_path) as img:
                # *** 10. 新增：Resize 步骤 ***
                img_resized = img.resize((IMAGE_SIZE, IMAGE_SIZE), resample=LANCZOS_RESAMPLE)
                
                # 转换为 NumPy 数组
                if img_resized.mode == 'RGBA':
                    img_resized = img_resized.convert('RGB')
                    
                img_array = np.array(img_resized)
                np.save(npy_path, img_array)
                return npy_path
        except Exception as e:
            warnings.warn(f" > 警告：处理 {jpg_path} 时出错: {e}。跳过。")
            return None

    non_feature_cols.extend(['npy_path'])
    
    tqdm.pandas(desc=" > 转换 训练集 图像")
    train_df['npy_path'] = train_df['absolute_image_path'].progress_apply(convert_jpg_to_npy)
    
    tqdm.pandas(desc=" > 转换 验证集 图像")
    val_df['npy_path'] = val_df['absolute_image_path'].progress_apply(convert_jpg_to_npy)
    
    tqdm.pandas(desc=" > 转换 测试集 图像")
    test_df['npy_path'] = test_df['absolute_image_path'].progress_apply(convert_jpg_to_npy)

    # 再次清洗：丢弃转换失败的图像
    train_df = train_df.dropna(subset=['npy_path'])
    val_df = val_df.dropna(subset=['npy_path'])
    test_df = test_df.dropna(subset=['npy_path'])
    
    print(" > 图像转换完成。")

    # --- 10, 11, 12. 💾 保存输出 ---
    print("[Final] 正在保存所有处理后的文件...")

    final_feature_columns = final_continuous_cols + final_categorical_cols
    
    data_splits = {
        'train': train_df,
        'val': val_df,
        'test': test_df
    }
    
    for split_name, df in data_splits.items():
        
        # (Step 10) 保存表格特征
        features_path = os.path.join(output_dir, f"{split_name}_features.csv")
        df[final_feature_columns].to_csv(features_path, index=False, header=False)
        
        # (Step 11) 保存标签
        labels_path = os.path.join(output_dir, f"{split_name}_labels.pt")
        labels_tensor = torch.tensor(df[label_column].values, dtype=torch.int64)
        torch.save(labels_tensor, labels_path)
        
        # (Step 12) 保存图像路径
        paths_path = os.path.join(output_dir, f"{split_name}_paths.pt")
        npy_path_list = df['npy_path'].tolist()
        torch.save(npy_path_list, paths_path)

    # (Step 10) 保存字段长度
    lengths_path = os.path.join(output_dir, "tabular_lengths.pt")
    torch.save(field_lengths, lengths_path)

    print("-" * 30)
    print("🎉 预处理全部完成！ 🎉")
    print(f"输出文件已保存到: {output_dir}")
    print(f" > 标签 (Label): {label_column}")
    print(f" > 表格特征: {len(final_feature_columns)} 个其他属性")
    print(f" > 划分比例: 8:1:1 (分层)")
    print(f" > 图像处理: 已缩放至 {IMAGE_SIZE}x{IMAGE_SIZE} 并转为 .npy")
    print("-" * 30)


if __name__ == "__main__":
    # 运行主预处理函数
    preprocess_celeba_flow(
        attr_file=ATTR_FILE,
        base_image_dir=BASE_IMAGE_DIR,
        output_dir=OUTPUT_DIR,
        label_column=LABEL_COLUMN
    )

开始执行 CelebA 预处理流程 (标签: Attractive)...
 正在加载标注文件 (list_attr_celeba.csv)...
 > 成功加载 202599 条记录。
 正在定义特征...
 > 已定义 0 个连续特征。
 > 已定义 40 个类别特征 (包含标签)。
 > 目标标签 (Label): Attractive
 正在按 8:1:1 对 Attractive 进行分层划分...
 > 训练集: 162079 样本
 > 验证集: 20260 样本
 > 测试集: 20260 样本
 正在筛选恒定特征...
 > 已将 'Attractive' 隔离为标签, 不作为特征。
 > 剩余 0 个连续特征。
 > 剩余 39 个类别特征 (表格属性)。
 正在转换图像路径为绝对路径...
 正在过滤无效行 (基于路径和标签)...
 > 训练集移除 0 行
 > 验证集移除 0 行
 > 测试集移除 0 行
 正在处理缺失值/特殊编码...
 > [CelebA 特定] 转换属性：将 -1 映射到 0...
 正在处理类别特征...
 > CelebA 属性已为 0/1 编码，跳过.cat.codes。
 > 正在计算基数 (Cardinalities)...


/tmp/ipykernel_292101/4247396634.py:196: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if "MISSING" in all_values.unique():


 > 39 个类别特征的基数 (部分): [2, 2, 2, 2, 2]...
 正在标准化连续特征...
 > 没有连续特征，跳过标准化。
 正在转换图像 (JPG -> NPY, 并 Resize 到 224x224)...


 > 转换 测试集 图像: 100%|██████████| 20260/20260 [00:00<00:00, 55325.04it/s]


 > 图像转换完成。
[Final] 正在保存所有处理后的文件...
------------------------------
🎉 预处理全部完成！ 🎉
输出文件已保存到: ./features
 > 标签 (Label): Attractive
 > 表格特征: 39 个其他属性
 > 划分比例: 8:1:1 (分层)
 > 图像处理: 已缩放至 224x224 并转为 .npy
------------------------------
